In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [215]:
def job_search(interest_rate=0.0435,c=107,urban_disposable_income=679,w_min=254,T=0,fee=300,inflator=1):
    urban_disposable_income=int(inflator*urban_disposable_income)
    w_min=int(inflator*w_min)
    beta=1/(1+interest_rate)
    w_max=1500  #in 2018, minimun wage in BJ was 2120 per month so w_min is 25.44 (2.12*12) but minimun wage is not disposable income
    w_num=w_max-w_min+1  

    tol=10e-4
    w_vec=np.linspace(w_min,w_max,w_num)

    #obtain the empirical log normal distribution
    dist_mean,dist_num=np.round(np.log(urban_disposable_income),1),1000000
    dist_std=np.round((dist_mean-np.log(w_min))/3,1) # suppose the lower bound is w_min, based on 3 sigma rule, need further investigation

    #obtain the empirical log normal distribution
    dist=np.random.lognormal(dist_mean,dist_std,dist_num)
    
    w_pdf=np.zeros(w_num)
    w_pdf[0]=np.sum(w_vec[0]>=dist)  #lognormal 是正无穷的，然而wage有下限，因此把下限的pdf设为小于下限的概率之和
    for ii in range(1,w_num-1):
        w_pdf[ii]=np.sum(np.logical_and(w_vec[ii-1]<dist,dist<=w_vec[ii]))
    w_pdf[-1]=np.sum(w_vec[-2]<dist)  #lognormal 是正无穷的，然而wage有上限，因此把上限的pdf设为 大于上限的概率之和
    w_pdf=w_pdf/dist_num
    
    tuition=fee*T  #assuming tuition per year is 30 thousand
    v_new=np.zeros(w_num)
    v_old=np.ones(w_num)
    while np.max(np.abs(v_new-v_old))>tol:
        v_old=np.copy(v_new)
        v_expected=np.sum(v_old*w_pdf)
        for w_i in range(w_num):
            v_new[w_i]=max(w_vec[w_i]*beta**T/(1-beta)-tuition,c+beta*v_expected)
    index=sum(v_new<=v_new[0])
    accept_rate=sum(dist>=w_vec[index])/dist_num
    #wage_scheme=np.copy(w_vec)
    #wage_scheme[:index]=c
    
    #return sum(wage_scheme*w_pdf)*100
    return accept_rate

In [220]:
job_search(T=0)

0.198757

In [221]:
job_search(inflator=1.04,T=2)

0.197174

In [222]:
job_search(inflator=1.14,T=4)

0.200519

In [225]:
job_search(inflator=1.26,T=6)

0.212535